# Annuities

## Annuities in Payment

In [ ]:
import os
import numpy as np
import pandas as pd

import protolinc
import protolinc.risk_factors as risk_factors

In [ ]:
table_path = os.path.abspath(os.path.join(protolinc.__path__[0], "..", "data", "tables"))
table_path

### DAV2004R
We look at this example from Germany. Annuities in payment are modelled using select factors, gender and age. Reasoning for the select effect is the option to select a lump sum payment at the begin of the annuity payment period.

In [ ]:
# read DAV2004R base table
dav_2004r_path = os.path.join(table_path, "Germany_Annuities_DAV2004R", "Germany_Annuities_DAV2004R.csv")

tab_DAV2004R_base = pd.read_csv(dav_2004r_path, header=[0, 2, 3], index_col=0)
tab_DAV2004R_base

In [ ]:
ages = np.sort(tab_DAV2004R_base.index.values.astype(np.int32))
ages

In [ ]:
# transform to tabular form
df_tmp = tab_DAV2004R_base.unstack()
df_tmp.index.names=["TABLE_TYPE", "ESTIMATE_TYPE", "GENDER", "AGE"]
df_tmp = df_tmp.reset_index().rename({0: "QX"}, axis=1)
df_tmp.GENDER = df_tmp.GENDER.map({"Männer": risk_factors.Gender.M, "Frauen": risk_factors.Gender.F})
df_tmp.ESTIMATE_TYPE = df_tmp.ESTIMATE_TYPE.map({'2. Ordnung': "BE", 'Bestand': "?", '1. Ordnung': "LOADED"})
df_tmp.TABLE_TYPE = df_tmp.TABLE_TYPE.map({'Selektionstafel': "SELECT", 'Aggregattafel': "AGGREGATE"})
df_tmp

In [ ]:
df_tmp.TABLE_TYPE.unique(), df_tmp.ESTIMATE_TYPE.unique()

In [ ]:
ages = np.sort(df_tmp.AGE.drop_duplicates())

In [ ]:
def base_table(table_type='AGGREGATE', estimate_type="BE"):
    df = df_tmp[(df_tmp.TABLE_TYPE == table_type) & (df_tmp.ESTIMATE_TYPE == estimate_type)]
    return df.set_index(['GENDER', 'AGE'])[["QX"]].unstack("GENDER").droplevel(0, axis=1)

tab = base_table(table_type='SELECT', estimate_type="BE")
tab

From the documentation we learn that a lookup for this table family requires the following parameters:

  * as *meta parameters*:
  
    * table type: *aggregate* or *select*
    * estimate type: *be* or *loaded*
    * the trend beginning and ending year offsets $T_1$ and $T_2$
  * *gender*
  * *age*
  * calendar year (to calculate the trend)
  * in case the selection table should be used the year of payment (1, 2-5, >5)

### DAV2004R Trends

In [ ]:
# read DAV2004R base table
dav_2004r_trend_path = os.path.join(table_path, "Germany_Annuities_DAV2004R", "Germany_Annuities_DAV2004R_Trends.csv")

tab_DAV2004R_trends = pd.read_csv(dav_2004r_trend_path, header=[0, 2, 3], index_col=0)

# transform to tabular form
df_tmp = tab_DAV2004R_trends.unstack()
df_tmp.index.names=["LONG/SHORT", "ESTIMATE_TYPE", "GENDER", "AGE"]
df_tmp = df_tmp.reset_index().rename({0: "F"}, axis=1)
df_tmp.GENDER = df_tmp.GENDER.map({"Männer": risk_factors.Gender.M, "Frauen": risk_factors.Gender.F})
df_tmp.ESTIMATE_TYPE = df_tmp.ESTIMATE_TYPE.str.strip().map({'2. Ordnung': "BE", 'Bestand': "?", '1. Ordnung': "LOADED"})
df_tmp["LONG/SHORT"] = df_tmp["LONG/SHORT"].map({
    'F_1(x)': "F1",
    'F_1(y)': "F1",
    'F_2(x)': "F2",
    'F_2(y)': "F2",
    'F(x)': "F",
    'F(y)': "F"
})
df_tmp

In [ ]:
# for processing select an estimate type:
df_tmp1 = df_tmp[df_tmp.ESTIMATE_TYPE == "BE"]\
    .set_index(["LONG/SHORT", "ESTIMATE_TYPE", "GENDER", "AGE"])\
    .unstack("GENDER")\
    .droplevel(0, axis=1)\
    .unstack("LONG/SHORT")\
    .droplevel(0, axis=0)

df_tmp1

In [ ]:
# for processing select an estimate type:
df_tmp2 = df_tmp[df_tmp.ESTIMATE_TYPE == "LOADED"]\
    .set_index(["LONG/SHORT", "ESTIMATE_TYPE", "GENDER", "AGE"])\
    .unstack("GENDER")\
    .droplevel(0, axis=1)\
    .unstack("LONG/SHORT")\
    .droplevel(0, axis=0)

df_tmp2

In [ ]:
class MortalityRatesProvider:
    """ Rates provides class is used to return the rates
        given selector array for each risk factor. """
    
    def __init__(self, values, risk_factors, offsets=None):
        """ Data to be provided is an n-dimensional numpy array and an iterable of RiskFactors."""
        self.num_dimensions = len(values.shape)
        self.offsets = offsets
        if offsets is not None:
            assert len(self.offsets) == self.num_dimensions
        else:
            self.offsets = [0] * self.num_dimensions
        assert self.num_dimensions == len(risk_factors), "Number of risk factors and dimension of values must agree!"
        assert self.num_dimensions >= 0 and self.num_dimensions < 4, "Number of dimensions must be between 0 and 3"
        self.values = np.copy(values)
        self.risk_factors = [rf.__name__.lower() for rf in risk_factors]
    
    def get_rates(self, **kwargs):
        """ Rates are returned, the arguments must agree with the risk factors and
            contain numpy arrays that are used to index the values tensor. """
        
        attr = {k.lower(): v for (k,v) in kwargs.items()}
        
        # check for unknown risk factors provided
        unknown_rfs = kwargs.keys() - set(self.risk_factors)
        if unknown_rfs:
            raise Exceptions("Risk factors unknows for this provider: " + str(unknown_rfs))
        
        # order the selectors
        selectors = []
        for rf_name in self.risk_factors:
            selectors.append(attr[rf_name])
            
        if self.num_dimensions == 0:
            return self.values[0]
        elif self.num_dimensions == 1:
            return self.values[selectors[0] - self.offsets[0]]
        elif self.num_dimensions == 2:
            return self.values[selectors[0] - self.offsets[0],
                               selectors[1] - self.offsets[1]]
        elif self.num_dimensions == 3:
            return self.values[selectors[0] - self.offsets[0],
                               selectors[1] - self.offsets[1],
                               selectors[2] - self.offsets[2]]
        
        raise Exception("Method must be implemented in subclass.")
    
    def __repr__(self):
        return "<MortalityRatesProvider with factors ({})>".format(str(self.risk_factors))

    
class DAV2004R:
    """ Represents the DAV2004R table family for annuities. The table is a generation type family
        with risk factors age, gender and (optionally) the select year.
    
        Providers can be obtained for:
        
          - best estimate ("BE", "2. Ordnung")
          - with safety loadings ("LOADED", "1. Ordnung")
    
    """
    def __init__(self, base_directory, trend_t1=10, trend_t2=25):
        self.base_rates_path = os.path.abspath(os.path.join(base_directory, "Germany_Annuities_DAV2004R.csv"))
        self.select_rates_path = os.path.abspath(os.path.join(base_directory, "Germany_Annuities_DAV2004R_Select.csv"))
        self.trend_rates_path = os.path.abspath(os.path.join(base_directory, "Germany_Annuities_DAV2004R_Trends.csv"))

        self.base_year_trend = 1999
        # trend_t1 and trend_t2 are used for the trend flattening
        self.trend_t1 = trend_t1
        self.trend_t2 = trend_t2
        
        # load and transform base rates
        tab_DAV2004R_base = pd.read_csv(self.base_rates_path, header=[0, 2, 3], index_col=0)
        df_tmp = tab_DAV2004R_base.unstack()
        df_tmp.index.names=["TABLE_TYPE", "ESTIMATE_TYPE", "GENDER", "AGE"]
        df_tmp = df_tmp.reset_index().rename({0: "QX"}, axis=1)
        df_tmp.GENDER = df_tmp.GENDER.map({"Männer": risk_factors.Gender.M, "Frauen": risk_factors.Gender.F})
        df_tmp.ESTIMATE_TYPE = df_tmp.ESTIMATE_TYPE.map({'2. Ordnung': "BE", 'Bestand': "?", '1. Ordnung': "LOADED"})
        df_tmp.TABLE_TYPE = df_tmp.TABLE_TYPE.map({'Selektionstafel': "SELECT", 'Aggregattafel': "AGGREGATE"})
        self.df_base_rates = df_tmp   # .copy()
        
        # extract the ages
        self.ages = np.sort(tab_DAV2004R_base.index.values)
        
        # load and transform the trends
        tab_DAV2004R_trends = pd.read_csv(self.trend_rates_path, header=[0, 2, 3], index_col=0)
        df_tmp = tab_DAV2004R_trends.unstack()
        df_tmp.index.names=["LONG/SHORT", "ESTIMATE_TYPE", "GENDER", "AGE"]
        df_tmp = df_tmp.reset_index().rename({0: "F"}, axis=1)
        df_tmp.GENDER = df_tmp.GENDER.map({"Männer": risk_factors.Gender.M, "Frauen": risk_factors.Gender.F})
        df_tmp.ESTIMATE_TYPE = df_tmp.ESTIMATE_TYPE.str.strip().map({'2. Ordnung': "BE", 'Bestand': "?", '1. Ordnung': "LOADED"})
        df_tmp["LONG/SHORT"] = df_tmp["LONG/SHORT"].map({
            'F_1(x)': "F1",
            'F_1(y)': "F1",
            'F_2(x)': "F2",
            'F_2(y)': "F2",
            'F(x)': "F",
            'F(y)': "F"
        })
        self.df_trend_rates = df_tmp   # .copy()
        
        
    def rates_provider(self, table_type='AGGREGATE', estimate_type="BE", t_begin=1999, t_end=2150):
        
        table_type = table_type.upper()
        estimate_type = estimate_type.upper()
        
        df_base = self.df_base_rates[(self.df_base_rates.TABLE_TYPE == table_type) & 
                                     (self.df_base_rates.ESTIMATE_TYPE == estimate_type)]
        df_base = df_base.set_index(['GENDER', 'AGE'])[["QX"]].unstack("GENDER").droplevel(0, axis=1)

        df_trend = self.df_trend_rates[self.df_trend_rates.ESTIMATE_TYPE == estimate_type]\
                       .set_index(["LONG/SHORT", "ESTIMATE_TYPE", "GENDER", "AGE"])\
                       .unstack("GENDER")\
                       .droplevel(0, axis=1)\
                       .unstack("LONG/SHORT")\
                       .droplevel(0, axis=0)
        
        table = None
        
        if estimate_type == "BE":
            table = _calculate_be_tables_with_trend(t_begin, t_end, self.trend_t1, self.trend_t2, df_base, df_trend)
        elif estimate_type == "LOADED":
            table = _calculate_loaded_tables_with_trend(t_begin, t_end, self.trend_t1, self.trend_t2, df_base, df_trend)
        else:
            raise Exception("Unknow estimate type: {}".format(estimate_type))
        
        if table_type == 'AGGREGATE':
            # base rates and trend for 'AGGREGATE'
            provider = MortalityRatesProvider(table,
                                              (risk_factors.CalendarYear, risk_factors.Gender, risk_factors.Age),
                                              offsets=(t_begin, 0, 0))
        elif table_type == 'SELECT':
            raise Exception("Not yet implemented.")
        
        return provider


def _G(t, T1, T2):
    """ Trend interpolation formula, cf.
        2018-01-24_DAV-Richtlinie_Herleitung_DAV2004R.pdf (online), p. 43
    """
    if t < 1999:
        raise Exception("Table not available for years before 1999")
    elif t <= 1999 + T1:
        return 1
    elif t <= 1999 + T2:
        return 1.0 - ((t - 1999 - T1) * (t - 1999 - T1 - 1)) / (2 * (T2 - T1) * (t - 1999))
    else:
        return (T1 + T2 + 1) / (2 * (t - 1999))

    
def _calculate_be_tables_with_trend(t_begin, t_end, T1, T2, df_base, df_trend):
    """ Only applicable for BE (not LOADED). Return a three dimensional array containing the mortality
        rates indexed by:
        
          - age
          - sex (index 0=M, 1=F)
          - calendar year (index 0 corresponds with t_begin)    
    """
    
    # create container for the result
    res_table = np.zeros((1 + t_end - t_begin, 2, df_base.shape[0]), dtype=np.float64)
    
    # base data males
    qx_male = df_base.values[:, risk_factors.Gender.M]
    F1x_male = df_trend[(risk_factors.Gender.M, "F1")].values
    F2x_male = df_trend[(risk_factors.Gender.M, "F2")].values
    
    # base data females
    qx_female = df_base.values[:, risk_factors.Gender.F]
    F1x_female = df_trend[(risk_factors.Gender.F, "F1")].values
    F2x_female = df_trend[(risk_factors.Gender.F, "F2")].values
    
    # trend factors
    for t in range(t_begin, 1 + t_end):
        g = _G(t, T1, T2)
        exponential_arg_male = (F2x_male + g * (F1x_male - F2x_male)) * (t - 1999)
        exponential_arg_female = (F2x_female + g * (F1x_female - F2x_female)) * (t - 1999)
        res_table[t - t_begin, 0, :] = qx_male * np.exp(-exponential_arg_male)
        res_table[t - t_begin, 1, :] = qx_female * np.exp(-exponential_arg_female)
    
    return res_table


def _calculate_loaded_tables_with_trend(t_begin, t_end, T1, T2, df_base, df_trend):
    """ Only applicable for LOADED (not BE)). Return a three dimensional array containing the mortality
        rates indexed by:
        
          - age
          - sex (index 0=M, 1=F)
          - calendar year (index 0 corresponds with t_begin)    
    """
    
    # create container for the result
    res_table = np.zeros((1 + t_end - t_begin, 2, df_base.shape[0]), dtype=np.float64)
    
    # base data males
    qx_male = df_base.values[:, risk_factors.Gender.M]
    Fx_male = df_trend[(risk_factors.Gender.M, "F")].values

    # base data females
    qx_female = df_base.values[:, risk_factors.Gender.F]
    Fx_female = df_trend[(risk_factors.Gender.F, "F")].values
    
    # trend factors
    for t in range(t_begin, 1 + t_end):
        exponential_arg_male = Fx_male * (t - 1999)
        exponential_arg_female = Fx_female * (t - 1999)
        res_table[t - t_begin, 0, :] = qx_male * np.exp(-exponential_arg_male)
        res_table[t - t_begin, 1, :] = qx_female * np.exp(-exponential_arg_female)
    
    return res_table


### Use Lib


In [11]:
import numpy as np
from protolinc.tables import DAV2004R
import protolinc.risk_factors as risk_factors

dav2004R = DAV2004R()

In [12]:


# a 20 year old male and a 30 year old female
ages = np.array([20, 30], dtype=np.int32)
genders = np.array([0, 1], dtype=np.int32)
calendaryear = np.array([1999, 1999], dtype=np.int32)

# two providers
dav2004R_provider_agg_be = dav2004R.rates_provider()
dav2004R_provider_agg_loaded = dav2004R.rates_provider(estimate_type="LOADED")

In [13]:
# aggregate BE
dav2004R_provider_agg_be.get_rates(age=ages, gender=genders, calendaryear=calendaryear)

array([0.000743, 0.000348])

In [14]:
# loaded BE
dav2004R_provider_agg_loaded.get_rates(age=ages, gender=genders, calendaryear=calendaryear)

array([0.000627, 0.000291])

In [15]:
dav2004R_provider_agg_loaded.get_rates(age=ages, gender=genders, calendaryear=calendaryear + 1)

array([0.00060662, 0.00028197])

In [16]:
# check some values
dav2004R_provider_agg_be.get_rates(calendaryear=1999, gender=risk_factors.Gender.M, age=np.arange(122))

array([4.07600e-03, 3.75000e-04, 2.53000e-04, 1.87000e-04, 1.45000e-04,
       1.28000e-04, 1.21000e-04, 1.04000e-04, 1.17000e-04, 1.00000e-04,
       9.80000e-05, 1.17000e-04, 1.23000e-04, 1.35000e-04, 1.65000e-04,
       2.28000e-04, 3.28000e-04, 4.32000e-04, 7.07000e-04, 7.47000e-04,
       7.43000e-04, 7.54000e-04, 7.41000e-04, 7.61000e-04, 7.37000e-04,
       7.31000e-04, 7.30000e-04, 7.43000e-04, 7.26000e-04, 7.15000e-04,
       7.09000e-04, 7.17000e-04, 7.42000e-04, 7.86000e-04, 8.45000e-04,
       9.18000e-04, 1.00800e-03, 1.11900e-03, 1.24200e-03, 1.36700e-03,
       1.49500e-03, 1.62600e-03, 1.75800e-03, 1.90000e-03, 2.05300e-03,
       2.21700e-03, 2.40000e-03, 2.60100e-03, 2.81300e-03, 3.03800e-03,
       3.27400e-03, 3.53400e-03, 3.80700e-03, 4.08800e-03, 4.36700e-03,
       4.63600e-03, 4.90100e-03, 5.17900e-03, 5.48500e-03, 5.84600e-03,
       6.28100e-03, 6.84800e-03, 7.56600e-03, 8.43800e-03, 9.43900e-03,
       1.05330e-02, 1.17790e-02, 1.33390e-02, 1.50380e-02, 1.686

In [17]:
# select for future ages of one person
dav2004R_provider_agg_be.get_rates(calendaryear=2022 + np.arange(122), gender=risk_factors.Gender.M, age=np.arange(122))

array([2.10561130e-03, 1.89167296e-04, 1.24689267e-04, 9.00879952e-05,
       6.82826052e-05, 5.89208244e-05, 5.44453734e-05, 4.57431127e-05,
       5.03031291e-05, 4.20267595e-05, 4.02595341e-05, 4.69834924e-05,
       4.82815618e-05, 5.17996389e-05, 6.18861795e-05, 8.35913503e-05,
       1.17548503e-04, 1.51336529e-04, 2.42100787e-04, 2.50042685e-04,
       2.43107933e-04, 2.41156198e-04, 2.37242761e-04, 2.48279671e-04,
       2.51174298e-04, 2.57428811e-04, 2.62800454e-04, 2.70731524e-04,
       2.65261456e-04, 2.60020279e-04, 2.55367350e-04, 2.55201289e-04,
       2.60925943e-04, 2.73390395e-04, 2.91027341e-04, 3.12698182e-04,
       3.38631738e-04, 3.70034435e-04, 4.04230736e-04, 4.38389567e-04,
       4.72774313e-04, 5.06647659e-04, 5.39629001e-04, 5.74565536e-04,
       6.12554561e-04, 6.54137406e-04, 7.01807827e-04, 7.55485482e-04,
       8.12776406e-04, 8.73461806e-04, 9.35332703e-04, 1.00078900e-03,
       1.06810205e-03, 1.13597861e-03, 1.20195245e-03, 1.26481391e-03,
      